In [1]:
import pandas as pd
import numpy as np
import cv2

TRACKING_COLUMN_NAMES = ['frame', 'track_id', 'type', 'truncated', 'occluded', 'alpha', 'bbox_left', 'bbox_top',
                         'bbox_right', 'bbox_bottom', 'height', 'width', 'length', 'pos_x', 'pos_y', 'pos_z', 'rot_y']

df = pd.read_csv('/home/swh/data/kitti/tracking/training/label_02/0000.txt', header=None, sep=' ')
df.columns = TRACKING_COLUMN_NAMES
df.head()

frame  track_id        type  truncated  occluded      alpha    bbox_left  \
0      0        -1    DontCare         -1        -1 -10.000000   219.310000   
1      0        -1    DontCare         -1        -1 -10.000000    47.560000   
2      0         0         Van          0         0  -1.793451   296.744956   
3      0         1     Cyclist          0         0  -1.936993   737.619499   
4      0         2  Pedestrian          0         0  -2.523309  1106.137292   

     bbox_top   bbox_right  bbox_bottom       height        width  \
0  188.490000   245.500000   218.560000 -1000.000000 -1000.000000   
1  195.280000   115.480000   221.480000 -1000.000000 -1000.000000   
2  161.752147   455.226042   292.372804     2.000000     1.823255   
3  161.531951   931.112229   374.000000     1.739063     0.824591   
4  166.576807  1204.470628   323.876144     1.714062     0.767881   

        length      pos_x     pos_y      pos_z     rot_y  
0 -1000.000000 -10.000000 -1.000000  -1.000000 -1.000000  
1 -1000.000000 -10.000000 -1.000000  -1.000000 -1.000000  
2     4.433886  -4.552284  1.858523  13.410495 -2.115488  
3     1.785241   1.640400  1.675660   5.776261 -1.675458  
4     0.972283   6.301919  1.652419   8.455685 -1.900245

In [2]:
df.loc[df.type.isin(['Van','Truck','Tram']),'type'] = 'Car'
df = df[df.type.isin(['Car','Cyclist','Pedestrian'])]
df

frame  track_id        type  truncated  occluded     alpha    bbox_left  \
2         0         0         Car          0         0 -1.793451   296.744956   
3         0         1     Cyclist          0         0 -1.936993   737.619499   
4         0         2  Pedestrian          0         0 -2.523309  1106.137292   
7         1         0         Car          0         0 -1.796862   294.898777   
8         1         1     Cyclist          0         0 -1.935205   745.017137   
...     ...       ...         ...        ...       ...       ...          ...   
1084    153        10         Car          0         2 -1.818856   680.294919   
1085    153        11         Car          0         2  1.864481   245.920800   
1086    153        12  Pedestrian          1         0  0.826456  1185.199080   
1087    153        13         Car          0         0  1.773993   344.361560   
1088    153        14         Car          0         2 -1.728662   652.362288   

        bbox_top   bbox_right  bbox_bottom    height     width    length  \
2     161.752147   455.226042   292.372804  2.000000  1.823255  4.433886   
3     161.531951   931.112229   374.000000  1.739063  0.824591  1.785241   
4     166.576807  1204.470628   323.876144  1.714062  0.767881  0.972283   
7     156.024256   452.199718   284.621269  2.000000  1.823255  4.433886   
8     156.393157   938.839722   374.000000  1.739063  0.824591  1.785241   
...          ...          ...          ...       ...       ...       ...   
1084  177.511028   842.313244   284.070033  1.524000  1.728591  3.894227   
1085  194.456182   394.817829   286.444967  1.444000  1.595116  3.791789   
1086  151.165841  1241.000000   348.552707  1.688000  0.800000  0.884000   
1087  188.772369   430.531955   248.482384  1.422414  1.512803  3.707634   
1088  183.789605   737.478033   246.613864  1.365956  1.508586  3.485915   

         pos_x     pos_y      pos_z     rot_y  
2    -4.552284  1.858523  13.410495 -2.115488  
3     1.640400  1.675660   5.776261 -1.675458  
4     6.301919  1.652419   8.455685 -1.900245  
7    -4.650955  1.766774  13.581085 -2.121565  
8     1.700640  1.640419   5.778596 -1.664456  
...        ...       ...        ...       ...  
1084  2.353367  1.622590  12.436503 -1.637280  
1085 -5.458963  1.908188  13.979427  1.497916  
1086  5.739732  1.500532   6.279632  1.543272  
1087 -6.033258  1.888008  19.788795  1.481180  
1088  1.955738  1.651867  17.818612 -1.622048  

[711 rows x 17 columns]

In [3]:
box = np.array(df.loc[2,['bbox_left','bbox_top','bbox_right','bbox_bottom']])
box

array([296.744956, 161.752147, 455.226042, 292.372804], dtype=object)

In [4]:
DETECTION_COLOR_DICT={'Car':(255,255,0),'Pedestrian':(0,226,255),'Cyclist':(141,40,255)}

In [ ]:
import cv2

frame = 120
image = cv2.imread('/home/swh/data/kitti/RawData/2011_09_26/2011_09_26_drive_005_sync/image_02/data/%010d.png'%frame)

#df[df['frame'] == 0]
boxes = np.array(df[df.frame==frame][['bbox_left','bbox_top','bbox_right','bbox_bottom']])
types = np.array(df[df.frame==frame]['type'])

#for box in boxes:
for typ,box in zip(types,boxes):
    top_left = int(box[0]),int(box[1])
    bottom_right=int(box[2]),int(box[3])
    cv2.rectangle(image,top_left,bottom_right,DETECTION_COLOR_DICT[typ],2)
    
cv2.imshow("image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
def draw_point_cloud(ax, title, axes=[0, 1, 2], xlim3d=None, ylim3d=None, zlim3d=None):
        """
        Convenient method for drawing various point cloud projections as a part of frame statistics.
        """
        ax.scatter(*np.transpose(velo_frame[:, axes]), s=point_size, c=velo_frame[:, 3], cmap='gray')
        ax.set_title(title)
        ax.set_xlabel('{} axis'.format(axes_str[axes[0]]))
        ax.set_ylabel('{} axis'.format(axes_str[axes[1]]))
        if len(axes) > 2:
            ax.set_xlim3d(*axes_limits[axes[0]])
            ax.set_ylim3d(*axes_limits[axes[1]])
            ax.set_zlim3d(*axes_limits[axes[2]])
            ax.set_zlabel('{} axis'.format(axes_str[axes[2]]))
        else:
            ax.set_xlim(*axes_limits[axes[0]])
            ax.set_ylim(*axes_limits[axes[1]])
        # User specified limits
        if xlim3d!=None:
            ax.set_xlim3d(xlim3d)
        if ylim3d!=None:
            ax.set_ylim3d(ylim3d)
        if zlim3d!=None:
            ax.set_zlim3d(zlim3d)
            
        for t_rects, t_type in zip(tracklet_rects[frame], tracklet_types[frame]):
            draw_box(ax, t_rects, axes=axes, color=colors[t_type])


In [ ]:
def draw_box(pyplot_axis, vertices, axes=[0, 1, 2], color='black'):
    """
    Draws a bounding 3D box in a pyplot axis.
    
    Parameters
    ----------
    pyplot_axis : Pyplot axis to draw in.
    vertices    : Array 8 box vertices containing x, y, z coordinates.
    axes        : Axes to use. Defaults to `[0, 1, 2]`, e.g. x, y and z axes.
    color       : Drawing color. Defaults to `black`.
    """
    vertices = vertices[axes, :]
    connections = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Lower plane parallel to Z=0 plane
        [4, 5], [5, 6], [6, 7], [7, 4],  # Upper plane parallel to Z=0 plane
        [0, 4], [1, 5], [2, 6], [3, 7]  # Connections between upper and lower planes
    ]
    for connection in connections:
        pyplot_axis.plot(*vertices[:, connection], c=color, lw=0.5)
